## Import

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import zscore, pearsonr
from statsmodels.tsa.stattools import grangercausalitytests
import jieba
from snownlp import SnowNLP
import tushare as ts
import akshare as ak
from statsmodels.tsa.stattools import adfuller, coint, kpss, grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import statsmodels.api as sm
from linearmodels.panel import PanelOLS, RandomEffects, compare
from linearmodels.iv import IV2SLS
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Package

In [2]:
def test_stationarity(series, alpha=0.05):
    # ADF检验
    adf_result = adfuller(series.dropna())
    print(f"ADF Statistic: {adf_result[0]:.4f}")
    print(f"ADF p-value: {adf_result[1]:.4f}")
    print("ADF结论: 序列平稳" if adf_result[1] < alpha else "ADF结论: 序列非平稳")

    # KPSS检验
    kpss_result = kpss(series.dropna())
    print(f"\nKPSS Statistic: {kpss_result[0]:.4f}")
    print(f"KPSS p-value: {kpss_result[1]:.4f}")
    print("KPSS结论: 序列趋势平稳" if kpss_result[1] > alpha else "KPSS结论: 序列存在单位根")

In [3]:
def winsorize_series(series, lower=0.01, upper=0.99):
    """
    对收益率序列进行缩尾处理
    :param series: pandas Series, 收益率序列
    :param lower: 下缩尾分位数（默认1%）
    :param upper: 上缩尾分位数（默认99%）
    :return: 缩尾后的Series
    """
    # 计算分位数边界
    lower_bound = series.quantile(lower)
    upper_bound = series.quantile(upper)

    # 执行缩尾（极端值替换为边界值）
    return series.clip(lower=lower_bound, upper=upper_bound)

In [4]:
def get_stock_data(stock_id, start_time, end_time):
    ## 个股数据

    pro = ts.pro_api('2876ea85cb005fb5fa17c809a98174f2d5aae8b1f830110a5ead6211')
    # 拉取数据

    # 拉取数据
    stock_daily_basic = pro.daily_basic(**{
        "ts_code": stock_id,
        "trade_date": "",
        "start_date": start_time,
        "end_date": end_time,
        "limit": "",
        "offset": ""
    }, fields=[
        "ts_code",
        "trade_date",
        "turnover_rate",
        "pe",
        "pb"
    ])

    # 拉取数据
    stock_daily = pro.daily(**{
        "ts_code": stock_id,
        "trade_date": "",
        "start_date": start_time,
        "end_date": end_time,
        "offset": "",
        "limit": ""
    }, fields=[
        "ts_code",
        "trade_date",
        "open",
        "high",
        "low",
        "close",
        "vol",
        "amount"
    ])

    if stock_daily.empty:
        print(f'{stock_id} stock 行情数据 is empty')
    if stock_daily_basic.empty:
        print(f'{stock_id} stock 每日指标 is empty')
    stock = stock_daily.merge(stock_daily_basic, how='left', on=['ts_code', 'trade_date'])
    return stock

In [5]:
def sentiment_analysis(text, sentiment_dict):
    words = list(jieba.cut(text))

    # 原始 counts = 0
    nums = 0
    for word in words:
        if word in sentiment_dict:
            nums += 1
    return nums

In [6]:
def SentimentIndex(df, stock):
    trade_date = stock['trade_date'].tolist()
    trade_date = sorted(trade_date)
    for i in range(len(trade_date) - 1):
        start_time = pd.to_datetime(trade_date[i].strftime('%Y%m%d') + ' ' + '15:00')
        end_time = pd.to_datetime((trade_date[i + 1].strftime('%Y%m%d') + ' ' + '15:00'))
        df_sentiment = df[(df['post_datetime'] > start_time) & (df['post_datetime'] <= end_time)]
        if df_sentiment.empty:
            continue
        else:
            sentiment_index = (sum(df_sentiment['pos_num']) - sum(df_sentiment['neg_num'])) / sum(
                df_sentiment['word_num'])
            stock.loc[stock['trade_date'] == pd.to_datetime(trade_date[i + 1]).strftime(
                '%Y%m%d'), 'sentiment_index'] = sentiment_index
            snownlp_index = (sum(df_sentiment['snownlp'])/len(df_sentiment['snownlp']))
            stock.loc[stock['trade_date'] == pd.to_datetime(trade_date[i+1]).strftime('%Y%m%d'), 'snownlp_index'] = snownlp_index
    return stock

In [7]:
txt_file = 'Sentiment dict plus.txt'

# 初始化分类容器
positive_words = []
negative_words = []

# 读取并处理文件
try:
    with open(txt_file, "r", encoding="utf-8") as file:
        for line_num, line in enumerate(file, 1):
            # 清理并分割行内容
            cleaned_line = line.strip()
            if not cleaned_line:
                continue  # 跳过空行

            # 分割词语和数值（兼容空格/制表符分隔）
            parts = cleaned_line.split(maxsplit=1)  # 最多分割一次
            if len(parts) != 2:
                print(f"第 {line_num} 行格式错误：{line}")
                continue

            word, score = parts
            try:
                score = int(score)
            except ValueError:
                print(f"第 {line_num} 行数值格式错误：{score}")
                continue

            # 分类存储
            if score == 1:
                positive_words.append(word)
            elif score == -1:
                negative_words.append(word)
            else:
                print(f"第 {line_num} 行发现无效数值：{score}")

except FileNotFoundError:
    print("错误：未找到文件 financial_words.txt")
except Exception as e:
    print(f"发生未知错误：{str(e)}")

# 输出统计结果
print(f"\n积极词汇（共 {len(positive_words)} 个）：")
print(", ".join(positive_words[:3]) + ("..." if len(positive_words) > 3 else ""))

print(f"\n消极词汇（共 {len(negative_words)} 个）：")
print(", ".join(negative_words[:3]) + ("..." if len(negative_words) > 3 else ""))


积极词汇（共 3645 个）：
安定, 安康, 帮助...

消极词汇（共 6177 个）：
败坏名声, 被没收的, 变节...


In [8]:
post_info = 'post_info.post_zssh000300_old.csv'
initial_df = pd.read_csv(post_info)

In [9]:
initial_df = initial_df[initial_df['post_url'].str.contains('zssh000300', na=False)].copy()

In [10]:
len(initial_df)

8969

In [11]:
initial_df['post_datetime'] = pd.to_datetime(initial_df["post_date"] + " " + initial_df["post_time"])
initial_df = initial_df.sort_values(by=['post_datetime'], ascending=True)

In [12]:
initial_df = initial_df.dropna()

In [13]:
# delete unwanted columns
df = initial_df.drop(columns=['_id', 'post_url', 'post_author', 'post_time'])
df.head(1)

,post_title,post_view,comment_num,post_date,post_datetime
10588,人工智能ETF159819行业紧紧把握科技创新发展趋势积极拥抱生成式人工智能相,135,0,2023-08-07,2023-08-07 12:31:00


In [14]:
df['post_date'] = pd.to_datetime(df['post_date'])

In [15]:
df['pos_num'] = df['post_title'].apply(sentiment_analysis, args=(positive_words,))
df['neg_num'] = df['post_title'].apply(sentiment_analysis, args=(negative_words,))
df['word_num'] = df['post_title'].apply(len)

Building prefix dict from C:\ProgramData\anaconda3\Lib\site-packages\jieba\dict.txt ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.2534399032592773 seconds.
Prefix dict has been built succesfully.


In [16]:
df['snownlp'] = df['post_title'].apply(lambda x: SnowNLP(x).sentiments)

In [17]:
df.head(2)

,post_title,post_view,comment_num,post_date,post_datetime,pos_num,neg_num,word_num,snownlp
10588,人工智能ETF159819行业紧紧把握科技创新发展趋势积极拥抱生成式人工智能相,135,0,2023-08-07,2023-08-07 12:31:00,2,0,39,0.999983
10587,对市场8月份人工智能ETF159819行情持乐观态度大概率会走出触底回升行情，,117,0,2023-08-07,2023-08-07 12:35:00,2,0,39,0.998474


In [18]:
ts.set_token('2876ea85cb005fb5fa17c809a98174f2d5aae8b1f830110a5ead6211')
pro = ts.pro_api()
# 拉取数据
m2_yoy = pro.cn_m(**{
    "m": "",
    "start_m": 202301,
    "end_m": 202505,
    "limit": "",
    "offset": ""
}, fields=[
    "month",
    "m2_yoy"
])
print(m2_yoy)



     month  m2_yoy
0   202503     7.0
1   202502     7.0
2   202501     7.0
3   202412     7.3
4   202411     7.1
5   202410     7.5
6   202409     6.8
7   202408     6.3
8   202407     6.3
9   202406     6.2
10  202405     7.0
11  202404     7.2
12  202403     8.3
13  202402     8.7
14  202401     8.7
15  202312     9.7
16  202311    10.0
17  202310    10.3
18  202309    10.3
19  202308    10.6
20  202307    10.7
21  202306    11.3
22  202305    11.6
23  202304    12.4
24  202303    12.7
25  202302    12.9
26  202301    12.6


In [19]:
m2_yoy = m2_yoy.sort_values(by='month', ascending=True)

In [20]:
m2_yoy['m2_yoy_adjusted'] = m2_yoy['m2_yoy'].rolling(window=12, center=True).mean()

In [21]:
m2_yoy['m2_yoy_lag_3'] = m2_yoy['m2_yoy_adjusted'].shift(3)

In [22]:
m2_yoy = m2_yoy.ffill()
m2_yoy = m2_yoy.bfill()

In [23]:
m2_yoy = m2_yoy[['month', 'm2_yoy_lag_3']].copy()
m2_yoy = m2_yoy.rename(columns={'m2_yoy_lag_3': 'm2_yoy'})

In [24]:
m2_yoy = m2_yoy.set_index('month')['m2_yoy'].to_dict()

In [25]:
m2_yoy['202504'] = m2_yoy['202503']

In [26]:
m2_yoy

{'202301': 11.258333333333333,
 '202302': 11.258333333333333,
 '202303': 11.258333333333333,
 '202304': 11.258333333333333,
 '202305': 11.258333333333333,
 '202306': 11.258333333333333,
 '202307': 11.258333333333333,
 '202308': 11.258333333333333,
 '202309': 11.258333333333333,
 '202310': 11.258333333333333,
 '202311': 10.933333333333335,
 '202312': 10.583333333333334,
 '202401': 10.216666666666667,
 '202402': 9.783333333333333,
 '202403': 9.4,
 '202404': 8.975,
 '202405': 8.608333333333333,
 '202406': 8.25,
 '202407': 7.958333333333333,
 '202408': 7.724999999999999,
 '202409': 7.483333333333333,
 '202410': 7.283333333333334,
 '202411': 7.141666666666667,
 '202412': 7.0,
 '202501': 6.891666666666667,
 '202502': 6.891666666666667,
 '202503': 6.891666666666667,
 '202504': 6.891666666666667}

In [27]:
## HS300指数日线数据
ts.set_token('2876ea85cb005fb5fa17c809a98174f2d5aae8b1f830110a5ead6211')
pro = ts.pro_api()

# 拉取数据
HS300 = pro.index_daily(**{
    "ts_code": "000300.SH",
    "trade_date": "",
    "start_date": 20230808,
    "end_date": 20250418,
    "limit": "",
    "offset": ""
}, fields=[
    "ts_code",
    "trade_date",
    "close",
    "open",
    "high",
    "low",
    "pre_close",
    "change",
    "pct_chg",
    "vol",
    "amount"
])
print(HS300)



       ts_code trade_date      close       open       high        low  \
0    000300.SH   20250418  3772.5230  3760.0425  3782.0443  3754.4072   
1    000300.SH   20250417  3772.2221  3755.0862  3779.1253  3749.5040   
2    000300.SH   20250416  3772.8204  3757.8757  3775.1661  3721.6025   
3    000300.SH   20250415  3761.2348  3756.4788  3764.4860  3737.8495   
4    000300.SH   20250414  3759.1422  3772.0407  3776.6344  3754.0330   
..         ...        ...        ...        ...        ...        ...   
405  000300.SH   20230814  3855.9061  3840.8872  3859.4718  3814.0711   
406  000300.SH   20230811  3884.2538  3977.9834  3977.9834  3884.2538   
407  000300.SH   20230810  3975.7166  3962.8569  3976.4637  3945.3869   
408  000300.SH   20230809  3967.5652  3967.3566  3982.1091  3961.8434   
409  000300.SH   20230808  3979.7322  3980.0423  4002.2904  3963.4632   

     pre_close   change  pct_chg          vol        amount  
0    3772.2221   0.3009   0.0080  102948509.0  1.731094e+08  

In [28]:
ts.set_token('2876ea85cb005fb5fa17c809a98174f2d5aae8b1f830110a5ead6211')
pro = ts.pro_api()

# 拉取数据
HS300_basic = pro.index_dailybasic(**{
    "trade_date": "",
    "ts_code": "000300.SH",
    "start_date": 20230808,
    "end_date": 20250418,
    "limit": "",
    "offset": ""
}, fields=[
    "ts_code",
    "trade_date",
    "total_mv",
    "float_mv",
    "turnover_rate",
    "pe",
    "pb",
    "total_share"
])
print(HS300_basic)



       ts_code trade_date      total_mv      float_mv  turnover_rate     pe  \
0    000300.SH   20250418  5.593480e+13  4.198855e+13           0.32  12.10   
1    000300.SH   20250417  5.586207e+13  4.197099e+13           0.38  12.09   
2    000300.SH   20250416  5.586239e+13  4.197110e+13           0.48  12.09   
3    000300.SH   20250415  5.550268e+13  4.172010e+13           0.41  12.01   
4    000300.SH   20250414  5.531078e+13  4.160352e+13           0.50  11.96   
..         ...        ...           ...           ...            ...    ...   
405  000300.SH   20230814  4.973195e+13  3.821329e+13           0.38  11.89   
406  000300.SH   20230811  5.007981e+13  3.848109e+13           0.41  11.97   
407  000300.SH   20230810  5.113598e+13  3.934532e+13           0.32  12.23   
408  000300.SH   20230809  5.099382e+13  3.926694e+13           0.31  12.19   
409  000300.SH   20230808  5.114554e+13  3.938285e+13           0.36  12.23   

       pb   total_share  
0    1.31  4.210865e+12  

In [29]:
HS300 = HS300.merge(HS300_basic, how='left', on=['ts_code', 'trade_date'])

In [30]:
HS300['trade_date'] = pd.to_datetime(HS300['trade_date'])

In [31]:
HS300 = HS300.sort_values(by='trade_date', ascending=True)

In [32]:
HS300['m2_yoy'] = HS300['trade_date'].apply(lambda x: m2_yoy[x.strftime('%Y%m')])

In [33]:
missing_values = HS300.isnull().sum()
missing_values

ts_code          0
trade_date       0
close            0
open             0
high             0
low              0
pre_close        0
change           0
pct_chg          0
vol              0
amount           0
total_mv         0
float_mv         0
turnover_rate    0
pe               0
pb               0
total_share      0
m2_yoy           0
dtype: int64

In [34]:
result = SentimentIndex(df, HS300)

In [35]:
result['agreement_index'] = result['sentiment_index'].apply(lambda x: 1 - np.sqrt(1 - x ** 2))

In [36]:
result = result[~result['sentiment_index'].isna()].reset_index(drop=True)

In [37]:
result['sentiment_index_lag_1'] = result['sentiment_index'].shift(1).fillna(0)

In [38]:
result['return'] = np.log(result['close'] / result['close'].shift(1)) * 100

In [39]:
result.head(2)

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,...,turnover_rate,pe,pb,total_share,m2_yoy,sentiment_index,snownlp_index,agreement_index,sentiment_index_lag_1,return
0,000300.SH,2023-08-09,3967.5652,3967.3566,3982.1091,3961.8434,3979.7322,-12.1670,-0.3057,94640269.0,...,0.31,12.19,1.4,4.065827e+12,11.258333,0.023569,0.634315,0.000278,0.000000,NaN
1,000300.SH,2023-08-10,3975.7166,3962.8569,3976.4637,3945.3869,3967.5652,8.1514,0.2055,97176849.0,...,0.32,12.23,1.4,4.065827e+12,11.258333,0.010249,0.580220,0.000053,0.023569,0.20524


In [40]:
result['return'] = result['return'].fillna(0)
result['volatility'] = (result['high'] - result['low']) / ((result['high'] + result['low']) / 2) * 100

In [41]:
result = result.rename(columns={'return': 'returns'}).copy()

In [42]:
result['returns_new'] = winsorize_series(result['returns'])
result['log_volatility'] = np.log(result['volatility'] + 1e-5).pipe(lambda x: (x - x.mean()) / x.std())
result['turnover_rate_new'] = result['turnover_rate'].apply(lambda x: np.log(1 + x))
result['pe_new'] = winsorize_series(result['pe'], lower=0.01, upper=0.99)
result['pb_new'] = winsorize_series(result['pb'], lower=0.01, upper=0.99)

In [43]:
result['vol'] = result['vol'] / 1e6  # (百万)

In [44]:
len(result)

409

In [45]:
result['volatility'].describe()

count    409.000000
mean       1.355402
std        0.908618
min        0.360828
25%        0.846813
50%        1.097468
75%        1.499679
max        8.332997
Name: volatility, dtype: float64

In [46]:
result['m2_yoy'].describe()

count    409.000000
mean       8.705949
std        1.614093
min        6.891667
25%        7.141667
50%        8.250000
75%       10.216667
max       11.258333
Name: m2_yoy, dtype: float64

In [47]:
data_processed = result[
    ['trade_date', 'sentiment_index', 'agreement_index', 'vol', 'returns_new', 'volatility', 'log_volatility', 'pe_new',
     'pb_new', 'turnover_rate_new', 'sentiment_index_lag_1', 'm2_yoy', 'snownlp_index']].copy()

In [48]:
data_processed.head(2)

,trade_date,sentiment_index,agreement_index,vol,returns_new,volatility,log_volatility,pe_new,pb_new,turnover_rate_new,sentiment_index_lag_1,m2_yoy,snownlp_index
0,2023-08-09,0.023569,0.000278,94.640269,0.00000,0.510217,-1.691940,12.19,1.4,0.270027,0.000000,11.258333,0.634315
1,2023-08-10,0.010249,0.000053,97.176849,0.20524,0.784584,-0.820835,12.23,1.4,0.277632,0.023569,11.258333,0.580220


In [49]:
for column in data_processed.columns:
    print(f"\n=== {column} 平稳性检验 ===")
    test_stationarity(data_processed[column])


=== trade_date 平稳性检验 ===
ADF Statistic: 5.9313
ADF p-value: 1.0000
ADF结论: 序列非平稳

KPSS Statistic: 3.2465
KPSS p-value: 0.0100
KPSS结论: 序列存在单位根

=== sentiment_index 平稳性检验 ===
ADF Statistic: -4.5751
ADF p-value: 0.0001
ADF结论: 序列平稳

KPSS Statistic: 0.2273
KPSS p-value: 0.1000
KPSS结论: 序列趋势平稳

=== agreement_index 平稳性检验 ===
ADF Statistic: -12.2284
ADF p-value: 0.0000
ADF结论: 序列平稳

KPSS Statistic: 0.1822
KPSS p-value: 0.1000
KPSS结论: 序列趋势平稳

=== vol 平稳性检验 ===
ADF Statistic: -2.0924
ADF p-value: 0.2476
ADF结论: 序列非平稳

KPSS Statistic: 1.2049
KPSS p-value: 0.0100
KPSS结论: 序列存在单位根

=== returns_new 平稳性检验 ===
ADF Statistic: -11.5261
ADF p-value: 0.0000
ADF结论: 序列平稳

KPSS Statistic: 0.1837
KPSS p-value: 0.1000
KPSS结论: 序列趋势平稳

=== volatility 平稳性检验 ===
ADF Statistic: -4.9723
ADF p-value: 0.0000
ADF结论: 序列平稳

KPSS Statistic: 0.2579
KPSS p-value: 0.1000
KPSS结论: 序列趋势平稳

=== log_volatility 平稳性检验 ===
ADF Statistic: -4.7732
ADF p-value: 0.0001
ADF结论: 序列平稳

KPSS Statistic: 0.2820
KPSS p-value: 0.1000
KPSS结论: 序列趋势平稳


C:\Users\Administrator\AppData\Local\Temp\ipykernel_39556\2839035517.py:9: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result = kpss(series.dropna())
C:\Users\Administrator\AppData\Local\Temp\ipykernel_39556\2839035517.py:9: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_result = kpss(series.dropna())
C:\Users\Administrator\AppData\Local\Temp\ipykernel_39556\2839035517.py:9: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_result = kpss(series.dropna())
C:\Users\Administrator\AppData\Local\Temp\ipykernel_39556\2839035517.py:9: InterpolationWarning: The test statistic is outside of the range of p-values availabl

In [50]:
for column in ['pe_new', 'pb_new', 'turnover_rate_new', 'vol', 'm2_yoy']:
    data_processed[column] = data_processed[column].diff()

In [51]:
for column in ['pe_new', 'pb_new', 'turnover_rate_new', 'vol', 'm2_yoy']:
    print(f"\n=== {column} 平稳性检验 ===")
    test_stationarity(data_processed[column])


=== pe_new 平稳性检验 ===
ADF Statistic: -8.8101
ADF p-value: 0.0000
ADF结论: 序列平稳

KPSS Statistic: 0.0991
KPSS p-value: 0.1000
KPSS结论: 序列趋势平稳

=== pb_new 平稳性检验 ===
ADF Statistic: -12.8356
ADF p-value: 0.0000
ADF结论: 序列平稳

KPSS Statistic: 0.1476
KPSS p-value: 0.1000
KPSS结论: 序列趋势平稳

=== turnover_rate_new 平稳性检验 ===
ADF Statistic: -7.6441
ADF p-value: 0.0000
ADF结论: 序列平稳

KPSS Statistic: 0.0761
KPSS p-value: 0.1000
KPSS结论: 序列趋势平稳

=== vol 平稳性检验 ===
ADF Statistic: -7.8461
ADF p-value: 0.0000
ADF结论: 序列平稳

KPSS Statistic: 0.0608
KPSS p-value: 0.1000
KPSS结论: 序列趋势平稳

=== m2_yoy 平稳性检验 ===
ADF Statistic: -20.8179
ADF p-value: 0.0000
ADF结论: 序列平稳

KPSS Statistic: 0.3147
KPSS p-value: 0.1000
KPSS结论: 序列趋势平稳


C:\Users\Administrator\AppData\Local\Temp\ipykernel_39556\2839035517.py:9: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_result = kpss(series.dropna())
C:\Users\Administrator\AppData\Local\Temp\ipykernel_39556\2839035517.py:9: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_result = kpss(series.dropna())
C:\Users\Administrator\AppData\Local\Temp\ipykernel_39556\2839035517.py:9: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_result = kpss(series.dropna())
C:\Users\Administrator\AppData\Local\Temp\ipykernel_39556\2839035517.py:9: InterpolationWarning: The test statistic is outside of the range of p-values availabl

In [52]:
data_processed = data_processed.fillna(0)

In [53]:
# 初步数据处理，标注化
# 1.1 Z-Score标准化（适用于收益率、波动率）
cols_to_standardize = ['returns_new', 'log_volatility', 'sentiment_index', 'sentiment_index_lag_1', 'volatility', 'snownlp_index']
data_std = data_processed[cols_to_standardize].apply(lambda x: (x - x.mean()) / x.std())

In [54]:
cols_to_minmax = []
# data_minmax = data[cols_to_minmax].apply(lambda x: (x - x.min())/(x.max() - x.min()))

In [55]:
data_processed = pd.concat([data_processed.drop(columns=cols_to_standardize + cols_to_minmax),
                            data_std], axis=1)

In [56]:
data_processed['log_volatility'].describe()

count    4.090000e+02
mean    -2.605902e-17
std      1.000000e+00
min     -2.393232e+00
25%     -6.663260e-01
50%     -1.414527e-01
75%      4.906389e-01
max      3.962350e+00
Name: log_volatility, dtype: float64

In [59]:
variables = ['returns_new', 'log_volatility', 'sentiment_index', 'pb_new', 'pe_new', 'turnover_rate_new', 'm2_yoy']

# 创建空的相关系数和 p 值矩阵
corr_matrix = pd.DataFrame(index=variables, columns=variables, dtype=float)
pval_matrix = pd.DataFrame(index=variables, columns=variables, dtype=float)

# 计算两两相关性与 p 值
for var1 in variables:
    for var2 in variables:
        corr, pval = pearsonr(data_processed[var1], data_processed[var2])
        corr_matrix.loc[var1, var2] = corr
        pval_matrix.loc[var1, var2] = pval

# 可视化输出结果
print("=== 相关系数矩阵 ===")
print(corr_matrix.round(3))

print("\n=== P值矩阵（显著性水平）===")
print(pval_matrix.round(3))

=== 相关系数矩阵 ===
                   returns_new  log_volatility  sentiment_index  pb_new  \
returns_new              1.000           0.224            0.165   0.838   
log_volatility           0.224           1.000            0.077   0.116   
sentiment_index          0.165           0.077            1.000   0.090   
pb_new                   0.838           0.116            0.090   1.000   
pe_new                   0.881           0.142            0.123   0.906   
turnover_rate_new        0.323           0.435            0.030   0.203   
m2_yoy                  -0.047          -0.034           -0.056  -0.011   

                   pe_new  turnover_rate_new  m2_yoy  
returns_new         0.881              0.323  -0.047  
log_volatility      0.142              0.435  -0.034  
sentiment_index     0.123              0.030  -0.056  
pb_new              0.906              0.203  -0.011  
pe_new              1.000              0.258  -0.037  
turnover_rate_new   0.258              1.000  -0.031  

In [58]:
significance_matrix = corr_matrix.round(3).astype(str) + pval_matrix.applymap(lambda p:
    '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.1 else '')
print(significance_matrix)

                      returns_new log_volatility sentiment_index    pb_new  \
returns_new                1.0***       0.224***        0.165***  0.838***   
log_volatility           0.224***         1.0***           0.077   0.116**   
sentiment_index          0.165***          0.077          1.0***     0.09*   
pb_new                   0.838***        0.116**           0.09*    1.0***   
pe_new                   0.881***       0.142***         0.123**  0.906***   
turnover_rate_new        0.323***       0.435***            0.03  0.203***   
m2_yoy                     -0.047         -0.034          -0.056    -0.011   
sentiment_index_lag_1       0.013         -0.045        0.168***    -0.017   
snownlp_index               0.077      -0.129***        0.412***     0.058   

                         pe_new turnover_rate_new  m2_yoy  \
returns_new            0.881***          0.323***  -0.047   
log_volatility         0.142***          0.435***  -0.034   
sentiment_index         0.123**     

C:\Users\Administrator\AppData\Local\Temp\ipykernel_39556\2060424128.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  significance_matrix = corr_matrix.round(3).astype(str) + pval_matrix.applymap(lambda p:


In [124]:
data_processed[['returns_new', 'log_volatility', 'sentiment_index', 'pb_new', 'pe_new', 'turnover_rate_new', 'm2_yoy', 'sentiment_index_lag_1','snownlp_index']].corr()

,returns_new,log_volatility,sentiment_index,pb_new,pe_new,turnover_rate_new,m2_yoy,sentiment_index_lag_1,snownlp_index
returns_new,1.000000,0.223819,0.165013,0.838278,0.880583,0.323064,-0.047296,0.012967,0.076805
log_volatility,0.223819,1.000000,0.076819,0.115505,0.141544,0.435001,-0.034320,-0.044703,-0.128523
sentiment_index,0.165013,0.076819,1.000000,0.089726,0.122560,0.030306,-0.055663,0.167548,0.412027
pb_new,0.838278,0.115505,0.089726,1.000000,0.906296,0.202773,-0.010927,-0.017291,0.057626
pe_new,0.880583,0.141544,0.122560,0.906296,1.000000,0.257529,-0.036617,-0.009070,0.064283
turnover_rate_new,0.323064,0.435001,0.030306,0.202773,0.257529,1.000000,-0.031053,-0.053413,-0.035518
m2_yoy,-0.047296,-0.034320,-0.055663,-0.010927,-0.036617,-0.031053,1.000000,0.087494,-0.065197
sentiment_index_lag_1,0.012967,-0.044703,0.167548,-0.017291,-0.009070,-0.053413,0.087494,1.000000,0.152018
snownlp_index,0.076805,-0.128523,0.412027,0.057626,0.064283,-0.035518,-0.065197,0.152018,1.000000


In [134]:
data_processed_pos = data_processed[data_processed['sentiment_index'] > 0].copy()

In [135]:
data_processed_neg = data_processed[data_processed['sentiment_index'] < 0].copy()

In [125]:
y_return = data_processed['returns_new']
X = sm.add_constant(data_processed[['snownlp_index', 'pe_new', 'pb_new', 'turnover_rate_new', 'm2_yoy']])

model_return_ols = sm.OLS(y_return, X).fit()

print("对数收益率 OLS 回归结果:")
print(model_return_ols.summary())

对数收益率 OLS 回归结果:
                            OLS Regression Results                            
Dep. Variable:            returns_new   R-squared:                       0.797
Model:                            OLS   Adj. R-squared:                  0.794
Method:                 Least Squares   F-statistic:                     316.3
Date:                Sat, 10 May 2025   Prob (F-statistic):          4.91e-137
Time:                        16:01:29   Log-Likelihood:                -253.86
No. Observations:                 409   AIC:                             519.7
Df Residuals:                     403   BIC:                             543.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 

In [127]:
y_return = data_processed['returns_new']
X = sm.add_constant(data_processed[['sentiment_index', 'pe_new', 'pb_new', 'turnover_rate_new', 'm2_yoy']])

model_return_ols = sm.OLS(y_return, X).fit()

print("对数收益率 OLS 回归结果:")
print(model_return_ols.summary())

对数收益率 OLS 回归结果:
                            OLS Regression Results                            
Dep. Variable:            returns_new   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.798
Method:                 Least Squares   F-statistic:                     322.7
Date:                Sat, 10 May 2025   Prob (F-statistic):          1.94e-138
Time:                        16:02:07   Log-Likelihood:                -250.57
No. Observations:                 409   AIC:                             513.1
Df Residuals:                     403   BIC:                             537.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 

In [136]:
y_volatility = data_processed_pos['log_volatility']
X = sm.add_constant(
    data_processed_pos[['snownlp_index', 'pe_new', 'pb_new', 'm2_yoy', 'turnover_rate_new']])

model_return_ols = sm.OLS(y_volatility, X).fit()

print("波动率 回归结果:")
print(model_return_ols.summary())

波动率 回归结果:
                            OLS Regression Results                            
Dep. Variable:         log_volatility   R-squared:                       0.298
Model:                            OLS   Adj. R-squared:                  0.280
Method:                 Least Squares   F-statistic:                     16.24
Date:                Sat, 10 May 2025   Prob (F-statistic):           2.47e-13
Time:                        16:56:48   Log-Likelihood:                -256.66
No. Observations:                 197   AIC:                             525.3
Df Residuals:                     191   BIC:                             545.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0428

In [141]:
y_volatility = data_processed_neg['log_volatility']
X = sm.add_constant(
    data_processed_neg[['sentiment_index', 'pe_new', 'pb_new', 'turnover_rate_new']])

model_return_ols = sm.OLS(y_volatility, X).fit()

print("波动率 回归结果:")
print(model_return_ols.summary())

波动率 回归结果:
                            OLS Regression Results                            
Dep. Variable:         log_volatility   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.243
Method:                 Least Squares   F-statistic:                     17.92
Date:                Sat, 10 May 2025   Prob (F-statistic):           1.20e-12
Time:                        17:00:11   Log-Likelihood:                -254.30
No. Observations:                 212   AIC:                             518.6
Df Residuals:                     207   BIC:                             535.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0152

In [95]:
y_volatility = data_processed['log_volatility']
X = sm.add_constant(
    data_processed[['sentiment_index', 'agreement_index', 'pe_new', 'pb_new', 'm2_yoy', 'turnover_rate_new']])

model_return_ols = sm.OLS(y_volatility, X).fit()

print("波动率 回归结果:")
print(model_return_ols.summary())

波动率 回归结果:
                            OLS Regression Results                            
Dep. Variable:         log_volatility   R-squared:                       0.195
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     16.21
Date:                Sat, 10 May 2025   Prob (F-statistic):           9.77e-17
Time:                        15:57:59   Log-Likelihood:                -535.52
No. Observations:                 409   AIC:                             1085.
Df Residuals:                     402   BIC:                             1113.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0184

In [96]:
# VIF计算
vif_data = pd.DataFrame()
X = data_processed[['sentiment_index_lag_1', 'pe_new', 'turnover_rate_new']].copy()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

In [97]:
vif_data

,Variable,VIF
0,sentiment_index_lag_1,1.002885
1,pe_new,1.071057
2,turnover_rate_new,1.074033


In [118]:
# Grange sentiment_index --> returns_new
test_data = data_processed[['returns_new', 'sentiment_index']].dropna()
granger_test = grangercausalitytests(test_data, maxlag=8)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=11.2962 , p=0.0009  , df_denom=405, df_num=1
ssr based chi2 test:   chi2=11.3799 , p=0.0007  , df=1
likelihood ratio test: chi2=11.2241 , p=0.0008  , df=1
parameter F test:         F=11.2962 , p=0.0009  , df_denom=405, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=12.0038 , p=0.0000  , df_denom=402, df_num=2
ssr based chi2 test:   chi2=24.3062 , p=0.0000  , df=2
likelihood ratio test: chi2=23.6081 , p=0.0000  , df=2
parameter F test:         F=12.0038 , p=0.0000  , df_denom=402, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=8.8222  , p=0.0000  , df_denom=399, df_num=3
ssr based chi2 test:   chi2=26.9310 , p=0.0000  , df=3
likelihood ratio test: chi2=26.0755 , p=0.0000  , df=3
parameter F test:         F=8.8222  , p=0.0000  , df_denom=399, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=6.6403  , p=0.0000  

In [119]:
# Grange returns_new --> sentiment_index
test_data = data_processed[['sentiment_index', 'returns_new']].dropna()
granger_test = grangercausalitytests(test_data, maxlag=8)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0004  , p=0.9849  , df_denom=405, df_num=1
ssr based chi2 test:   chi2=0.0004  , p=0.9848  , df=1
likelihood ratio test: chi2=0.0004  , p=0.9848  , df=1
parameter F test:         F=0.0004  , p=0.9849  , df_denom=405, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.6416  , p=0.5270  , df_denom=402, df_num=2
ssr based chi2 test:   chi2=1.2993  , p=0.5222  , df=2
likelihood ratio test: chi2=1.2972  , p=0.5228  , df=2
parameter F test:         F=0.6416  , p=0.5270  , df_denom=402, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.6865  , p=0.5607  , df_denom=399, df_num=3
ssr based chi2 test:   chi2=2.0956  , p=0.5528  , df=3
likelihood ratio test: chi2=2.0902  , p=0.5539  , df=3
parameter F test:         F=0.6865  , p=0.5607  , df_denom=399, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.9116  , p=0.4571  

In [120]:
# Grange sentiment --> log_volatility
test_data = data_processed[['log_volatility', 'sentiment_index']].dropna()
granger_test = grangercausalitytests(test_data, maxlag=8)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.6934  , p=0.4055  , df_denom=405, df_num=1
ssr based chi2 test:   chi2=0.6985  , p=0.4033  , df=1
likelihood ratio test: chi2=0.6979  , p=0.4035  , df=1
parameter F test:         F=0.6934  , p=0.4055  , df_denom=405, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.7657  , p=0.0641  , df_denom=402, df_num=2
ssr based chi2 test:   chi2=5.6002  , p=0.0608  , df=2
likelihood ratio test: chi2=5.5620  , p=0.0620  , df=2
parameter F test:         F=2.7657  , p=0.0641  , df_denom=402, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=2.4546  , p=0.0628  , df_denom=399, df_num=3
ssr based chi2 test:   chi2=7.4929  , p=0.0577  , df=3
likelihood ratio test: chi2=7.4246  , p=0.0595  , df=3
parameter F test:         F=2.4546  , p=0.0628  , df_denom=399, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=2.8397  , p=0.0241  

In [121]:
# Grange log_volatility --> sentiment
test_data = data_processed[['sentiment_index', 'log_volatility']].dropna()
granger_test = grangercausalitytests(test_data, maxlag=8)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.5199  , p=0.2184  , df_denom=405, df_num=1
ssr based chi2 test:   chi2=1.5312  , p=0.2159  , df=1
likelihood ratio test: chi2=1.5283  , p=0.2164  , df=1
parameter F test:         F=1.5199  , p=0.2184  , df_denom=405, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.2672  , p=0.2828  , df_denom=402, df_num=2
ssr based chi2 test:   chi2=2.5658  , p=0.2772  , df=2
likelihood ratio test: chi2=2.5578  , p=0.2783  , df=2
parameter F test:         F=1.2672  , p=0.2828  , df_denom=402, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=2.8699  , p=0.0363  , df_denom=399, df_num=3
ssr based chi2 test:   chi2=8.7607  , p=0.0326  , df=3
likelihood ratio test: chi2=8.6675  , p=0.0341  , df=3
parameter F test:         F=2.8699  , p=0.0363  , df_denom=399, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=2.0049  , p=0.0931  

### RESET检验(Ramsey) 检验是否遗漏变量

In [130]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan, linear_reset
from linearmodels.iv import IV2SLS

In [131]:
X = data_processed[['snownlp_index', 'pe_new', 'pb_new','turnover_rate_new', 'm2_yoy']]
X = sm.add_constant(X)
y = data_processed['returns_new']

# 拟合OLS模型
ols_model = sm.OLS(y, X).fit()
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:            returns_new   R-squared:                       0.797
Model:                            OLS   Adj. R-squared:                  0.794
Method:                 Least Squares   F-statistic:                     316.3
Date:                Sat, 10 May 2025   Prob (F-statistic):          4.91e-137
Time:                        16:43:46   Log-Likelihood:                -253.86
No. Observations:                 409   AIC:                             519.7
Df Residuals:                     403   BIC:                             543.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0021      0.02

In [143]:
X = data_processed[['sentiment_index', 'pe_new', 'pb_new','turnover_rate_new']]
X = sm.add_constant(X)
y = data_processed['returns_new']

# 拟合OLS模型
ols_model = sm.OLS(y, X).fit()
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:            returns_new   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.798
Method:                 Least Squares   F-statistic:                     403.8
Date:                Sat, 10 May 2025   Prob (F-statistic):          1.14e-139
Time:                        17:06:49   Log-Likelihood:                -250.80
No. Observations:                 409   AIC:                             511.6
Df Residuals:                     404   BIC:                             531.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0052      0.02

In [144]:
reset_test = linear_reset(ols_model, power=2, use_f=True)
print(f"RESET检验的p值为: {reset_test.pvalue}")

RESET检验的p值为: 0.0008974802545031172


In [145]:
# Breusch-Pagan 异方差检验
_, pval, _, _ = het_breuschpagan(ols_model.resid, X)
print(f"异方差检验p值: {pval}")

异方差检验p值: 2.1324630521945387e-11
